In [1]:
### DO NOT RUN THIS.  It builds the BkAvgAst... tables, which are supplied in the processing tables zip file.
### 20160510: both versions of BkAvgAst created (nohist & ultcert)
###           used Tickers from earlier version, then supplemented with missing Tickers if aeTotAst>800bn
### 20170407: updated for 2016-Dec updated FailedBank list and 2017-Mar institutions2.csv file

import numpy as np
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir('C:/BankRisk')


In [2]:
rk = pd.read_csv("BkAvgAstPR_FailDate_201612.csv", index_col=0)
rk['xAsOf'] = pd.to_datetime(rk['xAsOf'])
rk['nAsOf'] = pd.to_datetime(rk['nAsOf'])
rk['FailDate'] = pd.to_datetime(rk['FailDate'])
rk.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2016-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2016-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2016-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2016-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.395950e+09  1.369624e+09  4.808213e+08  6.097738e+08   
1 2001-03-31  1.210100e+09  1.216132e+09  6.332250e+08  7.478789e+08   
2 2001-03-31  1.021323e+09  1.017492e+09  4.930308e+08  2.606809e+08   
3 2001-03-31  7.900139e+08  7.775349e+08  4.884076e+08  5.403720e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.526110e+08  8.623848e+08        NaT  9.155524   9.155524  5.541019   
1  1.083941e+08  8.562730e+08 2009-01-16  7.936602  17.092126  7.297330   
2  4.333959e+08  6.940768e+08 2008-11-23  6.698479  23.790605  5.681723   
3  6.454305e+07  6.049150e+08        NaT  5.181410  28.972015  5.628445   
4  2.469325e+07  2.951088e+08        NaT  2.896596  31.868612  2.934454   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.541019  7.027077   7.027077  
1  12.838349  8.618610  15.645687  
2  18.520072  3.004105  18.649792  
3  24.148517  6.227286  24.877077  
4  27.082971  3.116289  27.993366

In [3]:
len(rk) # 10762

10762

In [4]:
bhc = pd.read_csv("INSTITUTIONS2_201703.csv", encoding='latin-1')
bhc.head()

STNAME  CERT  DOCKET  ACTIVE              ADDRESS      ASSET BKCLASS  \
0  Connecticut     4       0       0       22 Main Street     48,570      NM   
1  Connecticut     6       0       0  81 West Main Street    624,655      NM   
2        Maine     8       0       0      One City Center  1,699,404      SM   
3        Maine     9    9781       0       66 Main Street    539,169      SM   
4        Maine    10       0       0  8 Washington Street     55,692      SM   

   CHANGEC1  CHANGEC2  CHANGEC3 ...                                      CBSA  \
0       223         0         0 ...  Hartford-West Hartford-East Hartford, CT   
1       223       999         0 ...                     New Haven-Milford, CT   
2       223         0         0 ...               Portland-South Portland, ME   
3       223         0         0 ...                                       NaN   
4       223         0         0 ...               Portland-South Portland, ME   

   CBSA_NO                           CBSA_METRO_NAME CBSA_METRO  \
0    25540  Hartford-West Hartford-East Hartford, CT      25540   
1    35300                     New Haven-Milford, CT      35300   
2    38860               Portland-South Portland, ME      38860   
3        0                                       NaN          0   
4    38860               Portland-South Portland, ME      38860   

  CBSA_METRO_FLG CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0              1              0       NaN            0            0 NaN  
1              1              0       NaN            0            0 NaN  
2              1              0       NaN            0            0 NaN  
3              0              0       NaN            0            0 NaN  
4              1              0       NaN            0            0 NaN  

[5 rows x 132 columns]

In [5]:
bhc.replace(to_replace='orporated', value='', inplace=True, regex=True)
bhc.replace(to_replace='Irrevocable', value='Irrev', inplace=True, regex=True)
bhc.replace(to_replace='Employee Stock Ownership ', value='ESO', inplace=True, regex=True)
bhc.replace(to_replace='orporation', value='orp', inplace=True, regex=True)
bhc.replace(to_replace='inancial', value='in', inplace=True, regex=True)
bhc.replace(to_replace='Holding Company', value='HC', inplace=True, regex=True)
bhc.replace(to_replace=', Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=' Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=', The', value='', inplace=True, regex=True)

In [6]:
len(bhc) # 27596

27596

In [7]:
bhc.head(2)

STNAME  CERT  DOCKET  ACTIVE              ADDRESS    ASSET BKCLASS  \
0  Connecticut     4       0       0       22 Main Street   48,570      NM   
1  Connecticut     6       0       0  81 West Main Street  624,655      NM   

   CHANGEC1  CHANGEC2  CHANGEC3 ...                                      CBSA  \
0       223         0         0 ...  Hartford-West Hartford-East Hartford, CT   
1       223       999         0 ...                     New Haven-Milford, CT   

   CBSA_NO                           CBSA_METRO_NAME CBSA_METRO  \
0    25540  Hartford-West Hartford-East Hartford, CT      25540   
1    35300                     New Haven-Milford, CT      35300   

  CBSA_METRO_FLG CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0              1              0       NaN            0            0 NaN  
1              1              0       NaN            0            0 NaN  

[2 rows x 132 columns]

In [8]:
bhc2=bhc[['CERT','FED_RSSD','NAME','STALP','BKCLASS','OFFICES','NAMEHCR','RSSDHCR','STALPHCR','ULTCERT']]
bhc2.columns = [['Cert','IDRSSD','Name','ST','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']]
len(bhc2) # 27596

27596

In [9]:
bhc2.head(2)

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510

In [10]:
bhc2[bhc2['Parent']>' '][0:10]

Cert  IDRSSD                                 Name  ST BkClass  Offices  \
5     14   35301  State Street Bank and Trust Company  MA      SM      1.0   
15    35  749635                           AuburnBank  AL      SM      9.0   
19    39  430036            Robertson Banking Company  AL      NM      6.0   
21    41  432432                   Phenix-Girard Bank  AL      NM      4.0   
25    46  113937            Merchants Bank of Alabama  AL      NM      5.0   
26    47  390336               Traders & Farmers Bank  AL      NM      7.0   
29    50  470032                Community Spirit Bank  AL      NM      5.0   
30    51  382537                   The Bank of Vernon  AL      NM      2.0   
31    52  293437        The Citizens Bank of Winfield  AL      NM      1.0   
33    54  672537         The Exchange Bank of Alabama  AL      NM      5.0   

                               Parent    ParRSSD ParST  UltCert  
5                   State Street Corp  1111435.0    MA       14  
15        Auburn National Bancorp Inc  1129533.0    AL       35  
19                            Rbc Inc  2526755.0    AL       39  
21       Phenix-Girard Bancshares Inc  2013246.0    AL       41  
25         Merchants Fin Services Inc  1082955.0    AL       46  
26   Traders & Farmers Bancshares Inc  3467305.0    AL       47  
29  Independent Bancshares IncESOPlan  3587294.0    AL       50  
30        First Vernon Bancshares Inc  1139998.0    AL       51  
31   Citizens Bancorp Of Winfield Inc  1833616.0    AL       52  
33                       Gadsden Corp  1079067.0    AL       54

In [11]:
bhc2[(bhc2['ParRSSD']>0)&pd.isnull(bhc2['Parent'])]  ## should be null list

Empty DataFrame
Columns: [Cert, IDRSSD, Name, ST, BkClass, Offices, Parent, ParRSSD, ParST, UltCert]
Index: []

In [12]:
bhc2[(bhc2['UltCert']>0)&pd.isnull(bhc2['Parent'])].head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510  
2    NaN      NaN   NaN     3510  
3    NaN      NaN   NaN     4255  
4    NaN      NaN   NaN     3510

In [13]:
## extract ticker info from prior listing: (rkhn was expanded output from br001f_NewBHCtickers code)
rkh = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201612.csv", index_col=0)
rkhn = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201612n.csv", index_col=0)

len(rkh), len(rkhn) # 10762

(10762, 10762)

In [14]:
rkh.columns

Index(['Rank', 'IDRSSD', 'xAsOf', 'xName', 'xState', 'xCert', 'nAsOf',
       'FailDate', 'BkClass', 'Offices', 'Parent', 'ParRSSD', 'ParST',
       'UltCert', 'Ticker', 'aeTotAst', 'aaTotAst', 'aeTotLn', 'aeDDep',
       'aeFDep', 'aeTotDep', 'pr_eTA', 'pr_eTAcs', 'pr_eTL', 'pr_eTLcs',
       'pr_eDD', 'pr_eDDcs'],
      dtype='object')

In [15]:
len(rkh[rkh['Ticker']>'']) # count before new BHCs added

2883

In [16]:
len(rkhn[rkhn['Ticker']>'']) # count after new BHCs added

2905

In [19]:
rkhn[rkhn['Ticker']=='BFIN'] # this will be added by missing ticker file (later)

Empty DataFrame
Columns: [Rank, IDRSSD, xAsOf, xName, xState, xCert, nAsOf, FailDate, BkClass, Offices, Parent, ParRSSD, ParST, UltCert, Ticker, aeTotAst, aaTotAst, aeTotLn, aeDDep, aeFDep, aeTotDep, pr_eTA, pr_eTAcs, pr_eTL, pr_eTLcs, pr_eDD, pr_eDDcs]
Index: []

[0 rows x 27 columns]

In [20]:
#uex = hs3[['Date','UE','UE_'+st]].ix[hs3['UE'].idxmax()]
rkhn2 = rkhn[rkhn['ParRSSD']>0][['Rank','ParRSSD','ParST','Ticker']]
len(rkhn2) # 8646

8646

In [21]:
rkhn2.tail(2)

Rank    ParRSSD ParST Ticker
10758  10759  1094314.0    MO  CBCYB
10761  10762  3447567.0    TX    NaN

In [22]:
rkh3 = rkhn2.groupby(['ParRSSD','Ticker']).min()
rkh3.sort_index(by='Rank',inplace=True)
len(rkh3) # 417

417

In [23]:
rkh3.head()

Rank ParST
ParRSSD   Ticker            
1039502.0 JPM        1    NY
1073757.0 BAC        2    NC
1951350.0 C          3    NY
1120754.0 WFC        4    CA
1119794.0 USB        6    MN

In [24]:
rkh4 = rkh3.reset_index(drop=False)
rkh4.head()

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC
2  1951350.0      C     3    NY
3  1120754.0    WFC     4    CA
4  1119794.0    USB     6    MN

In [25]:
rkh5=rkh4[['ParRSSD','Ticker']].groupby('Ticker').count()
rkh5.head()

ParRSSD
Ticker         
ABCB          1
ABTX          1
ABYB          1
ACNB          1
ADKT          1

In [26]:
rkh5[rkh5['ParRSSD']>1] # finds tickers with more than one ParRSSD code:

Empty DataFrame
Columns: [ParRSSD]
Index: []

In [27]:
bhc2.head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510  
2    NaN      NaN   NaN     3510  
3    NaN      NaN   NaN     4255  
4    NaN      NaN   NaN     3510

In [28]:
bhc3 = bhc2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
bhc3.head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert Ticker  
0    NaN      NaN   NaN     3510    NaN  
1    NaN      NaN   NaN     3510    NaN  
2    NaN      NaN   NaN     3510    NaN  
3    NaN      NaN   NaN     4255    NaN  
4    NaN      NaN   NaN     3510    NaN

In [29]:
bhc3[bhc3['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert Ticker  
2233   4706.0  Bank Of America Corp  1073757.0    NC     3510    BAC

In [30]:
bhc2[bhc2['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert  
2233   4706.0  Bank Of America Corp  1073757.0    NC     3510

In [31]:
rk2 = rk.merge(bhc2[['Cert','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']],how='left',left_on='xCert',\
              right_on='Cert')
rk2.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2016-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2016-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn   ...      pr_eTLcs    pr_eDD  \
0  1.395950e+09  1.369624e+09  4.808213e+08   ...      5.541019  7.027077   
1  1.210100e+09  1.216132e+09  6.332250e+08   ...     12.838349  8.618610   

    pr_eDDcs    Cert  BkClass  Offices                Parent    ParRSSD  \
0   7.027077   628.0        N   5417.0  Jpmorgan Chase & Co.  1039502.0   
1  15.645687  3510.0        N   4706.0  Bank Of America Corp  1073757.0   

   ParST  UltCert  
0     NY    628.0  
1     NC   3510.0  

[2 rows x 27 columns]

In [32]:
rkh4.head(2)

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC

In [33]:
rk3 = rk2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
rk3.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2016-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2016-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn   ...      pr_eDD   pr_eDDcs  \
0  1.395950e+09  1.369624e+09  4.808213e+08   ...    7.027077   7.027077   
1  1.210100e+09  1.216132e+09  6.332250e+08   ...    8.618610  15.645687   

     Cert BkClass  Offices                Parent    ParRSSD  ParST  UltCert  \
0   628.0       N   5417.0  Jpmorgan Chase & Co.  1039502.0     NY    628.0   
1  3510.0       N   4706.0  Bank Of America Corp  1073757.0     NC   3510.0   

   Ticker  
0     JPM  
1     BAC  

[2 rows x 28 columns]

In [34]:
rk3c = list(rk3.columns)
rk3c2 = rk3c[0:7] + ['FailDate'] + rk3c[-7:] + rk3c[7:13] + rk3c[14:20]
rk3c, rk3c2

(['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'FailDate',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs',
  'Cert',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker'],
 ['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'FailDate',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs'])

In [35]:
rk4 = rk3[rk3c2]
rk4.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2016-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2016-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

    FailDate BkClass  Offices    ...           aeTotLn        aeDDep  \
0        NaT       N   5417.0    ...      4.808213e+08  6.097738e+08   
1 2009-01-16       N   4706.0    ...      6.332250e+08  7.478789e+08   

         aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
0  2.526110e+08  8.623848e+08  9.155524   9.155524  5.541019   5.541019   
1  1.083941e+08  8.562730e+08  7.936602  17.092126  7.297330  12.838349   

     pr_eDD   pr_eDDcs  
0  7.027077   7.027077  
1  8.618610  15.645687  

[2 rows x 27 columns]

In [36]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC_nohist201612.csv")

In [37]:
len(rk4) # 10762

10762

In [38]:
rkm = rk4[(rk4['ParRSSD']>0)&(rk4['aeTotAst']>700000)&pd.isnull(rk4['Ticker'])]
len(rkm) # 191

191

In [39]:
rkm.head()

Rank  IDRSSD      xAsOf                         xName xState    xCert  \
145   146  527954 2016-12-31  FIRST NATIONAL BANK OF OMAHA     NE   5452.0   
149   150  311845 2016-12-31                   ARVEST BANK     AR   8728.0   
162   163  137915 2016-12-31                 EMIGRANT BANK     NY  12054.0   
182   183  877369 2016-12-31                  RABOBANK, NA     CA  23364.0   
183   184  249612 2016-12-31        APPLE BANK FOR SAVINGS     NY  16068.0   

         nAsOf FailDate BkClass  Offices    ...           aeTotLn  \
145 2001-03-31      NaT       N    126.0    ...      7.053570e+06   
149 2001-03-31      NaT      SM    273.0    ...      6.452287e+06   
162 2001-03-31      NaT      SB      5.0    ...      5.207047e+06   
182 2001-03-31      NaT       N    108.0    ...      5.700340e+06   
183 2001-03-31      NaT      SB     80.0    ...      3.759378e+06   

           aeDDep  aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL  \
145  8.339843e+06     0.0  8.339843e+06  0.069367  73.960962  0.081286   
149  8.516992e+06  3596.0  8.520588e+06  0.066488  74.229182  0.074357   
162  6.415656e+06     0.0  6.415656e+06  0.060657  75.046399  0.060006   
182  5.675783e+06     0.0  5.675783e+06  0.053977  76.185468  0.065691   
183  6.999825e+06     0.0  6.999825e+06  0.053634  76.239102  0.043323   

      pr_eTLcs    pr_eDD   pr_eDDcs  
145  70.212938  0.096109  72.150419  
149  70.521400  0.098150  72.524887  
162  71.280566  0.073934  73.511618  
182  72.635957  0.065408  74.854570  
183  72.679280  0.080666  74.935236  

[5 rows x 27 columns]

In [40]:
rkm.to_csv('rk_missingtickers201612.csv')

In [44]:
#STOPPED HERE - 20170407

In [35]:
## check missingtickers file just created....
## filled in many missing tickers

In [41]:
rkm2 = pd.read_csv('rk_missingtickers201612_newtickers.csv',index_col=0)
rkm2.head()

Rank  IDRSSD       xAsOf                         xName xState  xCert  \
145   146  527954  12/31/2016  FIRST NATIONAL BANK OF OMAHA     NE   5452   
149   150  311845  12/31/2016                   ARVEST BANK     AR   8728   
162   163  137915  12/31/2016                 EMIGRANT BANK     NY  12054   
182   183  877369  12/31/2016                  RABOBANK, NA     CA  23364   
183   184  249612  12/31/2016        APPLE BANK FOR SAVINGS     NY  16068   

         nAsOf FailDate BkClass  Offices    ...          aeTotLn       aeDDep  \
145  3/31/2001      NaN       N      126    ...      7053570.438  8339843.234   
149  3/31/2001      NaN      SM      273    ...      6452287.094  8516991.984   
162  3/31/2001      NaN      SB        5    ...      5207046.781  6415655.563   
182  3/31/2001      NaN       N      108    ...      5700340.219  5675782.969   
183  3/31/2001      NaN      SB       80    ...      3759377.641  6999824.891   

     aeFDep     aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs    pr_eDD  \
145     0.0  8339843.234  0.069367  73.960962  0.081286  70.212938  0.096109   
149  3596.0  8520587.984  0.066488  74.229182  0.074357  70.521400  0.098150   
162     0.0  6415655.563  0.060657  75.046399  0.060006  71.280566  0.073934   
182     0.0  5675782.969  0.053977  76.185468  0.065691  72.635957  0.065408   
183     0.0  6999824.891  0.053634  76.239102  0.043323  72.679280  0.080666   

      pr_eDDcs  
145  72.150419  
149  72.524887  
162  73.511618  
182  74.854570  
183  74.935236  

[5 rows x 27 columns]

In [42]:
rkm3 = rkm2[rkm2['Ticker']>' '][['ParRSSD','Ticker']]
rkm3.head()
### WARNING:  If there are no rows here, then skip ahead to: rk4.to_csv("BkAvgAstPR_FailDate_BHC2_nohist201612.csv")

ParRSSD Ticker
639  3035928   BFIN

In [43]:
rkm3p = set(rkm3['ParRSSD'])
len(rkm3), len(rkm3p)

(1, 1)

In [44]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName        Parent Ticker
639  BANKFINANCIAL, NA  Bankfin Corp    NaN

In [45]:
rk4c = rk4.copy()


In [46]:
tdct = dict(zip(rkm3['ParRSSD'],rkm3['Ticker']))
tdct

{3035928: 'BFIN'}

In [47]:
## update missing Tickers if it exists:
for i in np.arange(len(rk4)):
  rs = rk4.ix[i]['ParRSSD']
  try:
    rk4.loc[i,'Ticker'] = tdct[rs]
  except:
    pass

In [48]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName        Parent Ticker
639  BANKFINANCIAL, NA  Bankfin Corp   BFIN

In [49]:
len(rk4[rk4['Ticker']>' ']),len(rk4c[rk4c['Ticker']>' ']) # 519, 517

(530, 529)

In [50]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC2_nohist201612.csv")

In [51]:
## process ultcert version:

In [52]:
rk5 = rk4.copy()

In [53]:
ult = set(rk5['UltCert'])
len(ult) # 6269

6190

In [54]:
bhcc = rk5[rk5['xCert'].isin(ult)][['xCert','Parent','Ticker','ParRSSD','ParST']]
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [55]:
bhcc.dropna(subset=['Parent'],inplace=True)
len(bhcc) # 4560

4508

In [56]:
len(bhcc[bhcc['Ticker']>'']) # 514

522

In [57]:
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [58]:
bhcc.groupby('xCert').filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

xCert                           Parent Ticker    ParRSSD ParST
39     16571.0                 Bank Of Montreal    BMO  1231333.0     0
212     4999.0              Cadence Bancorp Llc    NaN  4037349.0    TX
252    58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
546    26342.0             Royal Bank Of Canada     RY  1232497.0     0
643     4999.0              Cadence Bancorp Llc    NaN  4037349.0    TX
805    35525.0    Bankcap Partners Fund I, L.P.    NaN  3559657.0    TX
955    35525.0    Bankcap Partners Fund I, L.P.    NaN  3559657.0    TX
1116    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
1175   57197.0                      Bnccorp Inc   BNCC  1248162.0    ND
1353    3857.0     Bear State Fin Holdings, Llc    NaN  4297853.0    AR
1643   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
1689   16571.0                 Bank Of Montreal    BMO  1231333.0     0
1747    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
1750      89.0   White River Bancshares Company    NaN  3306589.0    AR
2089   58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
2149   57497.0           Pathfinder Bancorp Inc    NaN  2596776.0    NY
2536   26342.0             Royal Bank Of Canada     RY  1232497.0     0
2581   26575.0            South Shore Mutual HC    NaN  3919738.0    MA
3611   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4028   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
4202   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4345   26575.0            South Shore Mutual HC    NaN  3919738.0    MA
4378    3857.0     Bear State Fin Holdings, Llc    NaN  4297853.0    AR
4876   17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
5160   35274.0        Blue Ridge Bankshares Inc    NaN  1426960.0    VA
5409   11256.0          Pinnacle Bancshares Inc    NaN  3451416.0    AR
5645    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
5829    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
6423   11256.0          Pinnacle Bancshares Inc    NaN  3451416.0    AR
6864   17399.0                   First Fin Corp    NaN  1067514.0    KS
6968      89.0   White River Bancshares Company    NaN  3306589.0    AR
6976   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
7382   35274.0        Blue Ridge Bankshares Inc    NaN  1426960.0    VA
7431   13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
7548   15432.0               Uptown Bancorp Inc    NaN  1206797.0    SD
7667    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
7988    6080.0    John T. Vucurevich Foundation    NaN  3613504.0    SD
8687   57497.0           Pathfinder Bancorp Inc    NaN  2596776.0    NY
9054   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
9338    6080.0    John T. Vucurevich Foundation    NaN  3613504.0    SD
9612    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
9744   17399.0                   First Fin Corp    NaN  1067514.0    KS
10132  15432.0               Uptown Bancorp Inc    NaN  1206797.0    SD
10162  13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
10205  17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
10497  57197.0                      Bnccorp Inc   BNCC  1248162.0    ND

In [59]:
bhcc[bhcc['xCert']==17399]

xCert          Parent Ticker    ParRSSD ParST
6864  17399.0  First Fin Corp    NaN  1067514.0    KS
9744  17399.0  First Fin Corp    NaN  1067514.0    KS

In [60]:
len(bhcc) # 4508

4508

In [61]:
bhcc2 = bhcc.drop_duplicates(subset=['xCert','Parent','Ticker','ParRSSD'],keep=False)
len(bhcc2) # 4462

4462

In [62]:
bhcc2.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [63]:
bhcc2.set_index(['xCert'],drop=True,inplace=True)
bhcc2.head()

Parent Ticker    ParRSSD ParST
xCert                                                
628.0    Jpmorgan Chase & Co.    JPM  1039502.0    NY
3510.0   Bank Of America Corp    BAC  1073757.0    NC
7213.0          Citigroup Inc      C  1951350.0    NY
3511.0  Wells Fargo & Company    WFC  1120754.0    CA
6548.0           U.S. Bancorp    USB  1119794.0    MN

In [64]:
bhcc2.groupby(level=0).filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

Empty DataFrame
Columns: [Parent, Ticker, ParRSSD, ParST]
Index: []

In [65]:
dcrt = bhcc2[['Parent','Ticker','ParRSSD','ParST']].to_dict()
dcrt['Parent'][628]

'Jpmorgan Chase & Co.'

In [66]:
##### IMPORTANT CONDITIONAL PROCESSING FOR MISSING VALUES!!!!  ## improved 5/9/2016
##########################################################################
j = 0
rk5c = rk5.copy()
for i in np.arange(len(rk5c)):
  if (pd.isnull(rk5c.ix[i]['Parent']))|(rk5c.ix[i]['Parent']==' '):
    xc = rk5c.ix[i]['UltCert']
    try:
      rk5c.loc[i,'Parent'] = dcrt['Parent'][xc]
      rk5c.loc[i,'Ticker'] = dcrt['Ticker'][xc]
      rk5c.loc[i,'ParRSSD'] = dcrt['ParRSSD'][xc]
      rk5c.loc[i,'ParST'] = dcrt['ParST'][xc]
      j = j+1
    except:
      pass

In [67]:
len(rk5c[rk5c['Ticker']>'']) # 2932

2932

In [68]:
j # 4121

4121

In [69]:
len(rk5c) # 10762

10762

In [70]:
rk5c.to_csv("BkAvgAstPR_FailDate_BHC2_ult201612.csv")

In [ ]:
## demo of Zions affiliates (which were consolidated into one bank recently)

In [71]:
rk4[rk4['Ticker']=='ZION']  ## current single active affiliate

Rank  IDRSSD      xAsOf   xName xState   xCert      nAsOf FailDate  \
93    94  276579 2016-12-31  ZB, NA     UT  2270.0 2001-03-31      NaT   

   BkClass  Offices    ...          aeTotLn        aeDDep         aeFDep  \
93       N    468.0    ...      13196562.25  1.379843e+07  957346.546875   

        aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs    pr_eDD  \
93  1.475578e+07  0.122728  69.099161  0.152078  64.948604  0.159014   

     pr_eDDcs  
93  66.492437  

[1 rows x 27 columns]

In [72]:
rk5c[rk5c['Ticker']=='ZION']  ## all acquired & consolidated banks

Rank   IDRSSD      xAsOf                                 xName xState  \
93      94   276579 2016-12-31                                ZB, NA     UT   
146    147   837260 2015-09-30               CALIFORNIA BANK & TRUST     CA   
152    153   676656 2015-09-30                        AMEGY BANK, NA     TX   
287    288  1004368 2015-09-30              NATIONAL BANK OF ARIZONA     AZ   
316    317   456960 2015-09-30                     NEVADA STATE BANK     NV   
414    415   933957 2015-09-30              VECTRA BANK COLORADO, NA     NM   
682    683   807665 2009-06-30                     VINEYARD BANK, NA     CA   
1040  1041   458263 2006-12-31                  STOCKMEN'S BANK, THE     AZ   
1089  1090  1198641 2015-09-30  COMMERCE BANK OF WASHINGTON, NA, THE     WA   
1154  1155  2479310 2008-06-30                     SILVER STATE BANK     NV   
1561  1562   720465 2008-12-31                         ALLIANCE BANK     CA   
1621  1622   618656 2004-09-30                            KLEIN BANK     TX   
2638  2639   105950 2001-09-30          MINNEQUA BANK OF PUEBLO, THE     CO   
3084  3085   576158 2003-06-30                             MAXIMBANK     TX   
3988  3989   309664 2006-06-30                           RANCHO BANK     CA   
4075  4076   521550 2003-12-31                        LONE STAR BANK     TX   
4563  4564  2068134 2009-03-31            GREAT BASIN BANK OF NEVADA     NV   
6093  6094   626754 2001-03-31              LEAGUE CITY BANK & TRUST     TX   
6278  6279  3161993 2008-03-31                           CHOICE BANK     AZ   
6575  6576   687269 2007-06-30        INTERCONTINENTAL NATIONAL BANK     TX   
6616  6617   456670 2002-09-30                   FRONTIER STATE BANK     AZ   
8346  8347  3374186 2015-03-31          COMMERCE BANK OF OREGON, THE     OR   
9867  9868  2907448 2003-06-30               SOUTHLAND BUSINESS BANK     CA   

        xCert      nAsOf   FailDate BkClass  Offices     ...      \
93     2270.0 2001-03-31        NaT       N    468.0     ...       
146   20852.0 2001-03-31        NaT      NM     99.0     ...       
152   24107.0 2001-03-31        NaT       N     81.0     ...       
287   20626.0 2001-03-31        NaT       N     66.0     ...       
316   18113.0 2001-03-31        NaT      NM     50.0     ...       
414    2993.0 2001-03-31        NaT       N     37.0     ...       
682   23556.0 2001-03-31 2009-07-17       N     17.0     ...       
1040  23182.0 2001-03-31        NaT      NM     43.0     ...       
1089  27298.0 2001-03-31        NaT       N      2.0     ...       
1154  34194.0 2001-03-31 2008-09-05      NM     18.0     ...       
1561  23124.0 2001-03-31 2009-02-06      NM      5.0     ...       
1621  22866.0 2001-03-31        NaT      NM     27.0     ...       
2638   1773.0 2001-03-31        NaT      NM      5.0     ...       
3084   9017.0 2001-03-31        NaT      NM      9.0     ...       
3988  22934.0 2001-03-31        NaT      NM      4.0     ...       
4075  22304.0 2001-03-31        NaT      NM      6.0     ...       
4563  33824.0 2001-03-31 2009-04-17      NM      5.0     ...       
6093  17577.0 2001-03-31        NaT      NM      4.0     ...       
6278  57469.0 2003-03-31        NaT      NM      3.0     ...       
6575  19112.0 2001-03-31        NaT       N      3.0     ...       
6616  24227.0 2001-03-31        NaT      NM      7.0     ...       
8346  58223.0 2005-12-31        NaT      NM      1.0     ...       
9867  35287.0 2001-03-31        NaT      NM      1.0     ...       

           aeTotLn        aeDDep         aeFDep      aeTotDep    pr_eTA  \
93    1.319656e+07  1.379843e+07  957346.546875  1.475578e+07  0.122728   
146   7.675023e+06  8.519186e+06   32194.661017  8.551380e+06  0.067636   
152   6.722988e+06  7.001789e+06  646260.186441  7.648049e+06  0.064696   
287   3.325315e+06  3.496657e+06    2862.949153  3.499520e+06  0.027561   
316   2.461972e+06  3.219857e+06       0.000000  3.219857e+06  0.024247   
414   1.905816e+06  1.9167